In [41]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from dino_paint_utils import selfpredict_dino_forest
from FoodSeg103_data_handler import *
from scribbles_creator import create_even_scribbles
from convpaint_helpers import selfpred_convpaint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create scribbles

Load the images and ground truths as lists/batches

In [42]:
img_nums = [n for n in range(0, 4900, 1250)] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 4


Check and filter for resolution

In [43]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 200_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
1250:    65,536 pixels
2500:   196,608 pixels
3750:   241,664 pixels (!)


In [44]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] < thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 200,000: 3
   0: 196,608 pixels
1250:  65,536 pixels
2500: 196,608 pixels


Define scribbles parameters

In [45]:
bins=[0.125]
suff=["w3"] #standing for width 3
sq_scaling=100
scribble_width=3
scribbles_seed = 1

img, ground_truth = load_food_batch(img_nums)
folder_path = "./FoodSeg103_results/data/run01"

Loop and create scribbles

In [47]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        for s in suff:
            np.random.seed(scribbles_seed)
            print(f"IMG {img_num}: {bin}_{s}")
            scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, sq_scaling=sq_scaling, mode="all",
                                                            save_res=True, suff=s, show_res=False, image=None, print_steps=False, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"Annotation percentage: {perc_labelled:2.2f}%\n")

IMG 0: 0.125_w3
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 22 (11, 88)
Annotation percentage: 0.43%

IMG 1250: 0.125_w3
Adjusting square size and range to 12 (6, 50)
Adjusting square size and range to 12 (6, 50)
Adjusting square size and range to 12 (6, 50)
Adjusting square size and range to 12 (6, 50)
Adjusting square size and range to 12 (2, 50)
Adjusting square size and range to 6 (2, 50)
Adjusting square size and range to 3 (1, 50)
Adjusting square size and range to 12 (2, 50)
Adjusting square size and range to 6 (2, 50)
Adjusting square size and range to 3 (1, 50)
Adjusting square size and range to 12 (2, 50)
Adjusting square size and range to 6 (2, 50)
Adjusting square size and range to 3 (1, 50)
Adjusting squa

Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Show the last created scribble with its ground truth

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")